In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('NaturalL2').getOrCreate()

23/07/27 05:39:23 WARN Utils: Your hostname, codespaces-9c27ae resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/07/27 05:39:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/27 05:39:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [4]:
sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

In [5]:
sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish Java could...|
|  1.0|Logistic regressi...|
+-----+--------------------+



In [6]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

In [7]:
words_data = tokenizer.transform(sentenceData)

In [9]:
words_data.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+
|label|sentence                           |words                                     |
+-----+-----------------------------------+------------------------------------------+
|0.0  |Hi I heard about Spark             |[hi, i, heard, about, spark]              |
|0.0  |I wish Java could use case classes |[i, wish, java, could, use, case, classes]|
|1.0  |Logistic regression models are neat|[logistic, regression, models, are, neat] |
+-----+-----------------------------------+------------------------------------------+



In [10]:
hashing_tf = HashingTF(inputCol="words", outputCol="rawFeatures")

In [11]:
featurized_data = hashing_tf.transform(words_data)

In [12]:
idf = IDF(inputCol="rawFeatures", outputCol="features")

In [13]:
idf_model = idf.fit(featurized_data)

In [14]:
rescaled_data = idf_model.transform(featurized_data)

In [15]:
rescaled_data.select("label", "features").show(truncate=False)

23/07/27 05:59:25 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
23/07/27 05:59:26 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                      |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[18700,19036,33808,66273,173558],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                   |
|0.0  |(262144,[19036,20719,55551,58672,98717,109547,192310],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|1.0 

In [16]:
from pyspark.ml.feature import CountVectorizer

In [17]:
df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

In [18]:
df.show(truncate=False)

+---+---------------+
|id |words          |
+---+---------------+
|0  |[a, b, c]      |
|1  |[a, b, b, c, a]|
+---+---------------+



In [19]:
cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=3, minDF=2.0)

In [20]:
model = cv.fit(df)

In [21]:
result = model.transform(df)

In [22]:
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

